# Secção de Código

In [18]:
# ATENÇÃO À UTILIZAÇÃO DESTA FUNÇÃO
# DEMASIADAS QUERIES SEGUIDAS PODEM LEVAR A BLOQUEIO POR PARTE DO SERVIDOR
# EM PRINCÍPIO O MÓDULO ENTREZ ESTÁ PREPARADO PARA LIDAR COM ISSO MAS CONVÉM TER SEMPRE CUIDADO

def pesquisa_ncbi(email, term, db = 'pubmed', retmax = 10, rettype = 'abstract', retmode = 'text', display = 'Y' , save = 'N', ext='txt'):
    '''
    NOTA - PARA USAR RETMODE XML TÊM DE SER DECLARADA UMA NOVA VARIÁVEL POIS A FUNÇÃO AINDA NÃO CRIA FICHEIROS XML
    
    Função para a pesquisa de bibliografia na base de dados NCBI utilizando BioPython
    Esta função contempla um conjunto de parâmetros para seleção de base de dados e outputs desejado adaptando-se
    ao objetivo da pesquisa. 

    Por defeito a função pesquisa a base de dados pubmed e capta os 10 primeiros resultados para uma pesquisa
    generalizada. 

    Tem a opção de gerar um ficheiro de texto com os resultados da pesquisa de artigos e respetivos abstracts.

    No entanto a função pode ser especializada para uma pesquisa em base de dados de genes para captar ficheiros
    GenBank e FASTA para posterior análise com as funções específicas que estamos a desenvolver.

    Parâmetros
    ----------

    email : str
        email a ser utilizado pelo módulo Entrez para  pesquisa (e.g. exempl@mail.com)

    term : str
        termo a ser utilizado na pesquisa (e.g. 'gene-123 functions')

    db  : str
        base de dados onde irá ser realizada a pesquisa. Por defeito é utilizada a 'pubmed'
        Para efeitos deste trabalho iremos também utilizar: 'gene', 'nuccore' e 'protein'
        Outras opções disponíveis podem ser encontradas aqui: https://www.ncbi.nlm.nih.gov/books/NBK25497/table/chapter2.T._entrez_unique_identifiers_ui/?report=objectonly

    retmax : int
        define o número de resultados que queremos retirar para o output final (10 por defeito)

    rettype : str
        define o tipo de resultado que pretendemos retirar da base de dados ('abstract' por defeito)
        pode ser mudado para 'gb' para base de dados 'gene' ou 'fasta' para base de dados de nucleotidos ou proteina (sequencias)
        
    retmode : str
        define o retmode para a captura de resultados da pesquisa na base de dados. Por defeito 
        o retmode é text mas pode ser adaptado para 'xml' quando é pretendido obter um ficheiro xml (e.g base de dados nucleotide, ou genbank)

    display : str
        define se é pretendido o output da função legível (útil para quando se quer refinar a pesquisa antes/sem gravar o ficheiro)
        por defeito apresenta o resultado em formato legível
    
    save : str
        define se é pretendido gravar ou não o ficheiro de resultados na pasta 'lit-search-output'
        por defeito este parâmetro está inactivo

            
    
    '''
    # TODO criar ficheiros XML
    


    # Importação de módulos
    from Bio import Entrez
    import os

    # Definimos o email para os fins de pesquisa
    Entrez.email = email
        
    # Secção egquery que pode ser skipable

    handle = Entrez.egquery(term=term)
    egq_res = Entrez.read(handle)

    for _ in egq_res['eGQueryResult']:
        if db in _.values():
            if retmax > 1:
                print('Encontrados {} resultados em {}. Irão ser processados {} resultados.\n'.format(_['Count'],db,retmax))
            else:
                print('Encontrados {} resultados em {}. Irá ser processado 1 resultado.\n'.format(_['Count'],db))

    
    if rettype == 'fasta' or rettype == 'gb': retmax = 1

    # Sacar as IDs
    handle      = Entrez.esearch(db=db,term=term, retmode=retmode, retmax=retmax)
    esearch_res = Entrez.read(handle)

    lista_ids = esearch_res['IdList']
    # print('Top 10 artigos ->',lista_ids) # Converter para uma lista de títulos mais à frente'''
    
    

    # Transfere a informação
    handle = Entrez.efetch(db=db,id=','.join(lista_ids),rettype=rettype,retmode=retmode)
    fetch_res = handle.read()
    
    # Usar com print para já, para não estar a bagunçar demasiado.
    # Perceber como se pode implementar a escrita de ficheiros para ser mais fácil de mastigar os resultados.

    # NÃO FUNCIONA NO GITHUB / LIVECODING
    # ADICIONAR PARAMETRO save = 'Y' SE SE PRETENDER GUARDAR FICHEIRO 
    
    if display.upper() == 'Y':
        print(fetch_res,sep='\n')


    # Verifica a condição de gravação de output como ficheiro de texto
    
    if retmode == 'xml': save = 'n'
    
    if save.upper() == 'Y':  

        # Definimos a extensão em função da base de dados e retmode
        if rettype == 'gb':
            ext = '.gbk'
        
        elif rettype == 'fasta':
            ext = '.fasta'
        
        elif retmode == 'xml':
            ext = '.xml'

        else: ext = '.txt'

        filename = term+ext

        # Verifica se existe pasta para output e cria-a caso não exista
        cwd = os.getcwd()
        outputdir = os.path.join(cwd,'gene_search() output')
        if not os.path.exists(outputdir):
            os.mkdir(outputdir)


        # Muda o working directory para a pasta de output para gerar o ficheiro
        os.chdir(outputdir)

        if os.path.isfile(filename): print('Ficheiro já existe. Não será criado novo ficheiro.')



        # Gera o ficheiro           
        with open(term+ext , 'w', encoding='utf-8') as _:
            _.write(fetch_res)

        # Retorna ao working directory anterior
        os.chdir(cwd)

        print('Ficheiro gravado.')


    handle.close()

    print('Concluído.')

    return fetch_res

In [7]:
def parsing(nome_ficheiro):

    from Bio import SeqIO
    
    record = SeqIO.read(nome_ficheiro, "genbank")

    print("ID do gene:", record.id)

    print("Nome do gene:", record.name)

    print("Descrição do gene", record.description)

    print("Comprimento da sequência:", len(record.seq), "bp")

    return

In [8]:
def anot(nome_ficheiro):
    
    from Bio import SeqIO
    
    record = SeqIO.read(nome_ficheiro, "genbank")

    print("Quantidade de anotações:", len(record.annotations))

    print()
    
    print("Lista de anotações:")
    
    for anotacao in record.annotations:
        print(anotacao, "->", record.annotations[anotacao])

    return   


In [9]:
def features_qualifiers(nome_ficheiro):
    
    from Bio import SeqIO
    
    record = SeqIO.read(nome_ficheiro, "genbank")
    
    print("Quantidade de features:", len(record.features))

    for feature in record.features:
        print(feature)
    return


# Secção de resultados

## Literatura

In [10]:
email = 'pg21019@alunos.uminho.pt'
gene = 'hla-dqa2'
termos = [
    
    'hla-dqa1 hla-dqa2 hla-dqb1',
    'hla-dqa2 food allergy',
    'hla-dqa2 cancer'
]

In [11]:
# Pesquisa generalizada - top 10 resultaods

pesquisa_ncbi(email,gene,save='Y')

Encontrados 90 resultados em pubmed. Irão ser processados 10 resultados.

1. Hepatol Int. 2023 Nov 11. doi: 10.1007/s12072-023-10608-8. Online ahead of 
print.

Proteome-wide Mendelian randomization highlights AIF1 and HLA-DQA2 as targets 
for primary sclerosing cholangitis.

Chen L(1), Zhao Y(1), Li M(1), Lv G(2).

Author information:
(1)Department of Hepatobiliary and Pancreatic Surgery, General Surgery Center, 
First Hospital of Jilin University, No. 1 Xinmin Street, Changchun, 130021, 
Jilin, China.
(2)Department of Hepatobiliary and Pancreatic Surgery, General Surgery Center, 
First Hospital of Jilin University, No. 1 Xinmin Street, Changchun, 130021, 
Jilin, China. lvgy@jlu.edu.cn.

BACKGROUND: Primary sclerosing cholangitis (PSC) is a kind of cholestatic liver 
disease without effective therapies and its pathogenesis is largely unknown.
METHODS: We performed the proteome-wide Mendelian randomization (MR) design to 
estimate the causal associations of protein levels with PSC risk

"1. Hepatol Int. 2023 Nov 11. doi: 10.1007/s12072-023-10608-8. Online ahead of \nprint.\n\nProteome-wide Mendelian randomization highlights AIF1 and HLA-DQA2 as targets \nfor primary sclerosing cholangitis.\n\nChen L(1), Zhao Y(1), Li M(1), Lv G(2).\n\nAuthor information:\n(1)Department of Hepatobiliary and Pancreatic Surgery, General Surgery Center, \nFirst Hospital of Jilin University, No. 1 Xinmin Street, Changchun, 130021, \nJilin, China.\n(2)Department of Hepatobiliary and Pancreatic Surgery, General Surgery Center, \nFirst Hospital of Jilin University, No. 1 Xinmin Street, Changchun, 130021, \nJilin, China. lvgy@jlu.edu.cn.\n\nBACKGROUND: Primary sclerosing cholangitis (PSC) is a kind of cholestatic liver \ndisease without effective therapies and its pathogenesis is largely unknown.\nMETHODS: We performed the proteome-wide Mendelian randomization (MR) design to \nestimate the causal associations of protein levels with PSC risk. Therein, \ngenetic associations with 4,907 plasma pr

In [ ]:
# Pesquisa para os restantes termos
from time import sleep

for termo in termos:
    pesquisa_ncbi(email,termo,save='y',display='n')
    sleep(5)


In [12]:
# Identificação dos dados para pesquisa avançada
# Geração de ficheiros para processamento nas outras funções
# TODO alterar para save = 'y' no final
nucleotide = pesquisa_ncbi(email,gene,db='nuccore',save='n',retmax=1,display='n')          # Descobrir o identificador do gene
proteina   = pesquisa_ncbi(email,gene,db='protein',save='n',retmax=1,display='n') # Descobrir o identificador da proteína

Encontrados 99 resultados em nuccore. Irá ser processado 1 resultado.

Encontrados 57 resultados em protein. Irá ser processado 1 resultado.



In [13]:
id_nucleotide = nucleotide.split()[1]
id_proteina = proteina.split()[1]

In [14]:
print(id_nucleotide,id_proteina)

OR557191 DQB2_HUMAN


In [16]:
# Transferimos os ficheiros FASTA e GBK para uso futuro
pesquisa_ncbi(email,id_nucleotide,db='nuccore',retmax=1,rettype='gb',save='y',display='n')

Encontrados 1 resultados em nuccore. Irá ser processado 1 resultado.



'LOCUS       OR557191                5232 bp    DNA     linear   PRI 25-OCT-2023\nDEFINITION  Homo sapiens isolate PSCDA0402 MHC class II antigen (HLA-DQA2)\n            gene, HLA-DQA2*01:05:01:01extended allele, complete cds.\nACCESSION   OR557191\nVERSION     OR557191.1\nKEYWORDS    .\nSOURCE      Homo sapiens (human)\n  ORGANISM  Homo sapiens\n            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;\n            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;\n            Catarrhini; Hominidae; Homo.\nREFERENCE   1  (bases 1 to 5232)\n  AUTHORS   Khor,S.-S., Huijse,L., Endlich,S. and Tokunaga,K.\n  TITLE     Direct Submission\n  JOURNAL   Submitted (15-SEP-2023) Genome Medical Science Project, National\n            Center for Global Health and Medicine, Toyama 1-21-1, Shinjuku\n            Ward, Tokyo 162-8655, Japan\nCOMMENT     ##Assembly-Data-START##\n            Assembly Method       :: NGSengine v. 2.30.1.29498\n            Sequencing Technolog

In [20]:
pesquisa_ncbi(email,id_proteina,db='protein',retmax=1,rettype='fasta',save='y',display='n')

Encontrados 1 resultados em protein. Irá ser processado 1 resultado.

Ficheiro gravado.
Concluído.


'>sp|P05538.2|DQB2_HUMAN RecName: Full=HLA class II histocompatibility antigen, DQ beta 2 chain; AltName: Full=HLA class II histocompatibility antigen, DX beta chain; AltName: Full=MHC class II antigen DQB2; Flags: Precursor\nMSWKMALQIPGGFWAAAVTVMLVMLSTPVAEARDFPKDFLVQFKGMCYFTNGTERVRGVARYIYNREEYG\nRFDSDVGEFQAVTELGRSIEDWNNYKDFLEQERAAVDKVCRHNYEAELRTTLQRQVEPTVTISPSRTEAL\nNHHNLLVCSVTDFYPAQIKVRWFRNDQEETAGVVSTSLIRNGDWTFQILVMLEITPQRGDIYTCQVEHPS\nLQSPITVEWRAQSESAQSKMLSGIGGFVLGLIFLGLGLIIRHRGQKGPRGPPPAGLLH\n\n'

In [21]:
fich_gbk   = 'gene_search() output\\' + id_nucleotide + '.gbk'
fich_prot_fasta = 'gene_search() output\\' + id_proteina + '.fasta'

In [22]:
parsing(fich_gbk)

ID do gene: OR557191.1
Nome do gene: OR557191
Descrição do gene Homo sapiens isolate PSCDA0402 MHC class II antigen (HLA-DQA2) gene, HLA-DQA2*01:05:01:01extended allele, complete cds
Comprimento da sequência: 5232 bp


In [23]:
anot(fich_gbk)

Quantidade de anotações: 12

Lista de anotações:
molecule_type -> DNA
topology -> linear
data_file_division -> PRI
date -> 25-OCT-2023
accessions -> ['OR557191']
sequence_version -> 1
keywords -> ['']
source -> Homo sapiens (human)
organism -> Homo sapiens
taxonomy -> ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
references -> [Reference(title='Direct Submission', ...)]
structured_comment -> defaultdict(<class 'dict'>, {'Assembly-Data': {'Assembly Method': 'NGSengine v. 2.30.1.29498', 'Sequencing Technology': 'PacBio'}})


In [24]:
features_qualifiers(fich_gbk)

Quantidade de features: 13
type: source
location: [0:5232](+)
qualifiers:
    Key: db_xref, Value: ['taxon:9606']
    Key: isolate, Value: ['PSCDA0402']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Homo sapiens']

type: gene
location: [0:5232](+)
qualifiers:
    Key: allele, Value: ['HLA-DQA2*01:05:01:01extended']
    Key: gene, Value: ['HLA-DQA2']

type: mRNA
location: join{[0:162](+), [3796:4045](+), [4428:4710](+), [4877:5232](+)}
qualifiers:
    Key: allele, Value: ['HLA-DQA2*01:05:01:01extended']
    Key: gene, Value: ['HLA-DQA2']
    Key: product, Value: ['MHC class II antigen']

type: exon
location: [0:162](+)
qualifiers:
    Key: allele, Value: ['HLA-DQA2*01:05:01:01extended']
    Key: gene, Value: ['HLA-DQA2']
    Key: number, Value: ['1']

type: 5'UTR
location: [0:80](+)
qualifiers:
    Key: allele, Value: ['HLA-DQA2*01:05:01:01extended']
    Key: gene, Value: ['HLA-DQA2']

type: CDS
location: join{[80:162](+), [3796:4045](+), [4428:4710](+), [4877:50

## Análise BLAST

## Análise da proteína

In [30]:
from Bio import SeqIO
from Bio import ExPASy

fich_prot_fasta = 'gene_search() output\\' + id_proteina + '.fasta'

# Carregamos a sequência
seq_proteina = SeqIO.read(open(fich_prot_fasta),format='fasta')

In [32]:
# Procedemos à contagem de aminoácidos:
# Primeiro para a sequência traduzida e depois para o conjunto de sequências após a remoção de codões stop

from collections import Counter
contagem_amino_total = Counter(seq)

In [29]:
# Efetuamos uma pesquisa na base de dados SwissProt 
handle = ExPASy.get_sprot_raw(id_proteina)
seq_record = SeqIO.read(handle, "swiss")
id = seq_record.id
seq = seq_record.seq
tam = len(seq_record.seq)
name = seq_record.name
desc = seq_record.description
com = seq_record.annotations["comment"]
taxon = seq_record.annotations["taxonomy"]
organism = seq_record.annotations["organism"]
key = seq_record.annotations["keywords"]
print(f"ID {id} \n Sequência: {seq} \n Tamanho da sequência: {tam} bp")
print(f"Nome: {name} \n Descrição: {desc} \n Taxonomia: {taxon} \n Organismo: {organism} \n Keywords: {key}")

ID P05538 
 Sequência: MSWKMALQIPGGFWAAAVTVMLVMLSTPVAEARDFPKDFLVQFKGMCYFTNGTERVRGVARYIYNREEYGRFDSDVGEFQAVTELGRSIEDWNNYKDFLEQERAAVDKVCRHNYEAELRTTLQRQVEPTVTISPSRTEALNHHNLLVCSVTDFYPAQIKVRWFRNDQEETAGVVSTSLIRNGDWTFQILVMLEITPQRGDIYTCQVEHPSLQSPITVEWRAQSESAQSKMLSGIGGFVLGLIFLGLGLIIRHRGQKGPRGPPPAGLLH 
 Tamanho da sequência: 268 bp
Nome: DQB2_HUMAN 
 Descrição: RecName: Full=HLA class II histocompatibility antigen, DQ beta 2 chain; AltName: Full=HLA class II histocompatibility antigen, DX beta chain; AltName: Full=MHC class II antigen DQB2; Flags: Precursor; 
 Taxonomia: ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo'] 
 Organismo: Homo sapiens (Human) 
 Keywords: ['Adaptive immunity', 'Alternative splicing', 'Cell membrane', 'Disulfide bond', 'Endoplasmic reticulum', 'Endosome', 'Glycoprotein', 'Golgi apparatus', 'Immunity', 'Lysosome', 'Membrane', 'MHC II', 'Re

In [38]:
from Bio.Blast import NCBIWWW

handle = NCBIWWW.qblast('blastp','swissprot',open(fich_prot_fasta).read())

In [ ]:

from Bio.Blast import NCBIWWW
seq_prot = seq_record.seq
result_handle = NCBIWWW.qblast('blastp','swissprot', seq_prot)


In [ ]:
if not os.path.isfile('prot_blast_swiss.xml'):
    save_file = open("prot_blast_swiss.xml","w")
    save_file.write(result_handle.read())
    save_file.close()



In [ ]:
from Bio.Blast import NCBIXML
blast_record = NCBIXML.parse(open('prot_blast_swiss.xml'))

In [ ]:
from Bio import SearchIO
blast_records = SearchIO.read(open('prot_blast_swiss.xml'), "blast-xml")

In [ ]:
for br in blast_records:
    print(f'Sequence ID: {br.id}')
    print(f'Description: {br.description}')
    print(f'E-value: {br[0].evalue}')
    print(f'Bit Score: {br[0].bitscore}')
    print(f'Alignment:\n{br[0].aln}')
    print()

In [ ]:
prots = []
for p in proteina:
    if len(p) > 20:
        prots.append(p)
prots

In [ ]:
with open ("protein_seq.fasta","w") as file:
    file.write(f">Protein: \n {prots[:]}")

In [ ]:
from Bio import SeqIO
protein_seq = SeqIO.read("protein_seq.fasta","fasta")

In [ ]:
protein_seq.seq

In [ ]:
#Protein BLAST
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastp", "pdb", protein_seq)


from Bio import SearchIO
blast_records = SearchIO.read(result_handle, "blast-xml")

print(blast_records[:])

In [ ]:
from Bio import SearchIO
blast_records = SearchIO.read(result_handle, "blast-xml")

In [ ]:
from Bio.PDB.PDBParser import PDBParser
#Protein BLAST cdd
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastp", "CDD", seq_record.seq)


In [ ]:
from Bio import SearchIO
blast_records = SearchIO.read(result_handle, "blast-xml")
print(blast_records[:])

In [ ]:
for br in blast_records:
    print(f'Sequence ID: {br.id}')
    print(f'Description: {br.description}')
    print(f'E-value: {br[0].evalue}')
    print(f'Bit Score: {br[0].bitscore}')
    print(f'Alignment:\n{br[0].aln}')
    print()
prots = []
for p in proteina:
    if len(p) > 20:
        prots.append(p)
prots
with open ("protein_seq.fasta","w") as file:
    file.write(f">Protein: \n {prots[:]}")
from Bio import SeqIO
protein_seq = SeqIO.read("protein_seq.fasta","fasta")
protein_seq.seq
#Protein BLAST
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastp", "pdb", protein_seq)
from Bio import SearchIO
blast_records = SearchIO.read(result_handle, "blast-xml")
print(blast_records[:])

In [28]:
pesquisa_ncbi(email,gene+' protein',db='protein',rettype='fasta',retmax=1,save='y')

Encontrados 57 resultados em protein. Irá ser processado 1 resultado.

>sp|P05538.2|DQB2_HUMAN RecName: Full=HLA class II histocompatibility antigen, DQ beta 2 chain; AltName: Full=HLA class II histocompatibility antigen, DX beta chain; AltName: Full=MHC class II antigen DQB2; Flags: Precursor
MSWKMALQIPGGFWAAAVTVMLVMLSTPVAEARDFPKDFLVQFKGMCYFTNGTERVRGVARYIYNREEYG
RFDSDVGEFQAVTELGRSIEDWNNYKDFLEQERAAVDKVCRHNYEAELRTTLQRQVEPTVTISPSRTEAL
NHHNLLVCSVTDFYPAQIKVRWFRNDQEETAGVVSTSLIRNGDWTFQILVMLEITPQRGDIYTCQVEHPS
LQSPITVEWRAQSESAQSKMLSGIGGFVLGLIFLGLGLIIRHRGQKGPRGPPPAGLLH




'>sp|P05538.2|DQB2_HUMAN RecName: Full=HLA class II histocompatibility antigen, DQ beta 2 chain; AltName: Full=HLA class II histocompatibility antigen, DX beta chain; AltName: Full=MHC class II antigen DQB2; Flags: Precursor\nMSWKMALQIPGGFWAAAVTVMLVMLSTPVAEARDFPKDFLVQFKGMCYFTNGTERVRGVARYIYNREEYG\nRFDSDVGEFQAVTELGRSIEDWNNYKDFLEQERAAVDKVCRHNYEAELRTTLQRQVEPTVTISPSRTEAL\nNHHNLLVCSVTDFYPAQIKVRWFRNDQEETAGVVSTSLIRNGDWTFQILVMLEITPQRGDIYTCQVEHPS\nLQSPITVEWRAQSESAQSKMLSGIGGFVLGLIFLGLGLIIRHRGQKGPRGPPPAGLLH\n\n'

In [1]:
filename = 'gene_search() output\\hla-dqa2 protein.fasta'


In [5]:
from Bio import Seq, SeqIO

sequencia = SeqIO.read(open(filename),format='fasta')



False